## Import Credentials

In [0]:
%run "./Config"

In [0]:
# Mounting Directory for Yelp JSONs Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/unemploymentin"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/unemploymentin has been unmounted.
Out[1]: True

In [0]:
from pyspark.sql.functions import udf,col, when, count, isnan

In [0]:
 df= spark.read\
   .format("com.crealytics.spark.excel")\
   .option("header", "true")\
   .load(mount_point+"/ststdsadata.xlsx")
    



In [0]:
df = df.withColumnRenamed("States and selected areas:  Employment status of the civilian noninstitutional population,", "Code")
df = df.withColumnRenamed("_c1", "State")
df = df.withColumnRenamed("_c2", "Year")
df = df.withColumnRenamed("_c3", "Month")
df = df.withColumnRenamed("_c4", "PopulationCivil")
df = df.withColumnRenamed("_c5", "Total")
df = df.withColumnRenamed("_c6", "Percemt of Pop")
df = df.withColumnRenamed("_c7", "Total2")
df = df.withColumnRenamed("_c8", "Percent2")
df = df.withColumnRenamed("_c9", "UnemploymentTotal")
df = df.withColumnRenamed("_c10", "UnemploymentRate")



In [0]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----+----+-----+---------------+-----+--------------+------+--------+-----------------+----------------+
Code|State|Year|Month|PopulationCivil|Total|Percemt of Pop|Total2|Percent2|UnemploymentTotal|UnemploymentRate|
+----+-----+----+-----+---------------+-----+--------------+------+--------+-----------------+----------------+
 5| 6| 4| 5| 6| 5| 6| 4| 6| 4| 5|
+----+-----+----+-----+---------------+-----+--------------+------+--------+-----------------+----------------+

In [0]:
df = df.filter(col('Code') != '037')
df = df.filter(col('Code') != '51000')

In [0]:
df = df.select(['State','Year','Month','UnemploymentRate'])

In [0]:
df = df.na.drop("any")
df = df.filter(col('Year') >= 2000)
df.show(n=55)

+--------------------+----+-----+----------------+
 State|Year|Month|UnemploymentRate|
+--------------------+----+-----+----------------+
 Alabama|2000| 01| 4.7 |
 Alaska|2000| 01| 6.2 |
 Arizona|2000| 01| 4.1 |
 Arkansas|2000| 01| 4.3 |
 California|2000| 01| 5.0 |
 Colorado|2000| 01| 2.8 |
 Connecticut|2000| 01| 2.2 |
 Delaware|2000| 01| 3.4 |
District of Columbia|2000| 01| 5.7 |
 Florida|2000| 01| 3.7 |
 Georgia|2000| 01| 3.6 |
 Hawaii|2000| 01| 4.6 |
 Idaho|2000| 01| 4.6 |
 Illinois|2000| 01| 4.3 |
 Indiana|2000| 01| 3.1 |
 Iowa|2000| 01| 2.5 |
 Kansas|2000| 01| 3.6 |
 Kentucky|2000| 01| 4.1 |
 Louisiana|2000| 01| 4.9 |
 Maine|2000| 01| 3.7 |
 Maryland|2000| 01| 3.3 |
 Massachusetts|2000| 01| 3.1 |
 Michigan|2000| 01| 3.3 |
 Minnesota|2000| 01| 2.9 |
 Mississippi|2000| 01| 5.4 |
 Missouri|2000| 01| 3.0 |
 Montana|2000| 01| 5.0 |
 Nebraska|2000| 01| 2.7 |
 Nevada|2000| 01| 3.9 |
 New Hampshire|2000| 01| 2.8 |
 New Jersey|2000| 01| 3.8 |
 New Mexico|2000| 01| 5.2 |
 New York|2000| 01| 4.7 |
 North Carolina|2000| 01| 3.4 |
 North Dakota|2000| 01| 3.0 |
 Ohio|2000| 01| 4.2 |
 Oklahoma|2000| 01| 3.1 |
 Oregon|2000| 01| 5.0 |
 Pennsylvania|2000| 01| 4.2 |
 Rhode Island|2000| 01| 4.2 |
 South Carolina|2000| 01| 4.0 |
 South Dakota|2000| 01| 2.5 |
 Tennessee|2000| 01| 3.8 |
 Texas|2000| 01| 4.7 |
 Utah|2000| 01| 3.1 |
 Vermont|2000| 01| 2.8 |
 Virginia|2000| 01| 2.6 |
 Washington|2000| 01| 4.8 |
 West Virginia|2000| 01| 5.8 |
 Wisconsin|2000| 01| 3.1 |
 Wyoming|2000| 01| 4.0 |
 Alabama|2000| 02| 4.7 |
 Alaska|2000| 02| 6.2 |
 Arizona|2000| 02| 4.0 |
 Arkansas|2000| 02| 4.3 |
+--------------------+----+-----+----------------+
only showing top 55 rows

In [0]:
from pyspark.sql.types import IntegerType, FloatType

df = df.withColumn("Year", df["Year"].cast(IntegerType()))
df = df.withColumn("Month", df["Month"].cast(IntegerType()))
df = df.withColumn("UnemploymentRate", df["UnemploymentRate"].cast(FloatType()))

In [0]:
state = spark.read.format('csv').options(header='False').load(mount_point+'/name-abbr.csv')
state = state.withColumnRenamed("_c0", "State")
state = state.withColumnRenamed("_c1", "StateAbbreviation")
state.dtypes

Out[10]: [('State', 'string'), ('StateAbbreviation', 'string')]

In [0]:
df = df.join(state,["State"])

In [0]:
states = ['AZ','CA', 'DE', 'FL', 'ID', 'IL', 'IN', 'LA', 'MO', 'NV', 'NJ', 'PA', 'TN']
df = df.filter(df.StateAbbreviation.isin(states))
df = df.drop(col("State"))
df.dtypes

Out[12]: [('Year', 'int'),
 ('Month', 'int'),
 ('UnemploymentRate', 'float'),
 ('StateAbbreviation', 'string')]

In [0]:
df.write.format('json').mode('overwrite').save(
    mount_point+'/Unemployment_cleaned.json')

In [0]:
# write to the Unemployment table
try:
    df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
        .mode("append") \
        .option("dbtable", 'dbo.Unemployment') \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')